Data Clean Stage 1

In [1]:
import pandas as pd
import numpy as np
import re

# 1) Load
infile = "cardekho_used_cars_nagpur_price_fixed.xlsx"
df = pd.read_excel(infile)

# 2) Standardize column names (strip and snake_case)
df.columns = (
    df.columns
      .astype(str)
      .str.strip()
      .str.replace(r"\s+", "_", regex=True)
      .str.lower()
)

# 3) Drop fully empty rows and duplicate rows
df = df.dropna(how="all")
df = df.drop_duplicates()

# 4) Remove obvious non-car “header/bucket” rows that crept in
#    Heuristics based on your file preview: rows like "Under ₹2 Lakh", "Budget Cars ₹2 Lakh-₹5 Lakh", etc.
def looks_like_bucket_row(row):
    car_name = str(row.get("car_name", "")).strip()
    brand = str(row.get("brand", "")).strip()
    model = str(row.get("model", "")).strip()
    # patterns typical to section/bucket rows
    bucket_keywords = [
        "under ₹", "budget", "cars ₹", "suv cars", "cng cars", "automatic cars",
        "as good as new", "luxury cars", "electric cars", "above ₹", "₹2 -", "₹3 -", "₹5 -",
        "₹8 -", "₹10", "under", "above"
    ]
    text = " ".join([car_name.lower(), brand.lower(), model.lower()])
    return any(k in text for k in bucket_keywords)

if "car_name" in df.columns:
    mask_bucket = df.apply(looks_like_bucket_row, axis=1)
    df = df[~mask_bucket].copy()

# 5) Clean kms_driven: keep numbers only
if "kms_driven" in df.columns:
    df["kms_driven"] = (
        df["kms_driven"]
        .astype(str)
        .str.replace(r"[^\d]", "", regex=True)
        .replace({"": np.nan})
        .astype(float)
    )

# 6) Clean year_of_manufacture: numeric only
if "year_of_manufacture" in df.columns:
    df["year_of_manufacture"] = (
        df["year_of_manufacture"]
        .astype(str)
        .str.extract(r"(\d{4})")[0]
        .astype(float)
    )

# 7) Price cleaning:
#   - Examples in your sheet: "₹14.50 Lakh Compare Hyderabad View Seller Details"
#   - Keep only numeric lakh value like "14.50 Lakh", or convert to numeric_lakh
#   - Also normalize variants: "Rs.", "₹", spacing, and remove tails like "Compare ..."

def extract_price_lakh(text):
    """
    Return numeric price in lakh as float if found, else np.nan.
    Accepts patterns like:
      ₹14.50 Lakh, Rs. 7 Lakh, 6.2 Lakh
      46 Lakh, 1.94 Lakh
    Also handles '₹2 Lakh-₹5 Lakh' by taking the first numeric if such rows survived.
    """
    if pd.isna(text):
        return np.nan
    s = str(text)

    # Normalize currency symbols and spaces
    s = s.replace("Rs.", "").replace("Rs", "").replace("₹", "")
    s = re.sub(r"\s+", " ", s).strip()

    # Common case: number followed by 'Lakh'
    m = re.search(r"(\d+(?:\.\d+)?)\s*Lakh", s, flags=re.IGNORECASE)
    if m:
        return float(m.group(1))

    # If only a bare number appears and is large, assume it might be in lakh if explicitly stated elsewhere
    # but to be strict per your instruction, only keep values explicitly in 'Lakh'
    return np.nan

if "price" in df.columns:
    df["price_lakh"] = df["price"].apply(extract_price_lakh)

    # Keep a clean textual form like "14.50 Lakh" only when price_lakh exists
    df["price_clean"] = df["price_lakh"].apply(lambda x: (f"{x:.2f} Lakh") if pd.notna(x) else np.nan)

    # If you want strictly the cleaned text column replacing original 'price'
    df["price"] = df["price_clean"]

    # Optional: also keep a numeric column for analysis
    # df["price_lakh"] already exists

# 8) Remove rows with nulls in critical fields
critical_cols = [c for c in ["car_name", "brand", "model", "kms_driven", "fuel_type", "year_of_manufacture", "price"] if c in df.columns]
if critical_cols:
    df = df.dropna(subset=critical_cols, how="any")

# 9) Strip text fields and standardize casing for categorical columns
text_cols = [c for c in df.columns if df[c].dtype == "object"]
for c in text_cols:
    df[c] = df[c].astype(str).str.strip()

# Optional normalization for fuel_type labels
if "fuel_type" in df.columns:
    df["fuel_type"] = (
        df["fuel_type"]
        .str.title()  # Petrol, Diesel, Cng, Electric, etc.
        .replace({"Cng": "CNG"})  # common fix
    )

# 10) Reorder columns for readability
preferred_order = [col for col in ["car_name", "brand", "model", "year_of_manufacture", "fuel_type", "kms_driven", "price", "price_lakh"] if col in df.columns]
other_cols = [c for c in df.columns if c not in preferred_order]
df = df[preferred_order + other_cols]

# 11) Save outputs
df.to_excel("cardekho_used_cars_nagpur_price_clean.xlsx", index=False)
df.to_csv("cardekho_used_cars_nagpur_price_clean.csv", index=False)
print(f"Rows after cleaning: {len(df)}")


Rows after cleaning: 507


Data Clean Stage 2

In [3]:
import pandas as pd
import numpy as np
import re

# 1) Load
infile = "cardekho_used_cars_nagpur_price_clean.xlsx"

df = pd.read_excel(infile)

# 2) Standardize column names (strip and snake_case)
df.columns = (
    df.columns
      .astype(str)
      .str.strip()
      .str.replace(r"\s+", "_", regex=True)
      .str.lower()
)

# 3) Drop fully empty rows and duplicate rows
df = df.dropna(how="all")
df = df.drop_duplicates()

# 4) Remove obvious non-car “header/bucket” rows that crept in
#    Heuristics based on your file preview: rows like "Under ₹2 Lakh", "Budget Cars ₹2 Lakh-₹5 Lakh", etc.
def looks_like_bucket_row(row):
    car_name = str(row.get("car_name", "")).strip()
    brand = str(row.get("brand", "")).strip()
    model = str(row.get("model", "")).strip()
    # patterns typical to section/bucket rows
    bucket_keywords = [
        "under ₹", "budget", "cars ₹", "suv cars", "cng cars", "automatic cars",
        "as good as new", "luxury cars", "electric cars", "above ₹", "₹2 -", "₹3 -", "₹5 -",
        "₹8 -", "₹10", "under", "above"
    ]
    text = " ".join([car_name.lower(), brand.lower(), model.lower()])
    return any(k in text for k in bucket_keywords)

if "car_name" in df.columns:
    mask_bucket = df.apply(looks_like_bucket_row, axis=1)
    df = df[~mask_bucket].copy()

# 5) Clean kms_driven: keep numbers only
if "kms_driven" in df.columns:
    df["kms_driven"] = (
        df["kms_driven"]
        .astype(str)
        .str.replace(r"[^\d]", "", regex=True)
        .replace({"": np.nan})
        .astype(float)
    )

# 6) Clean year_of_manufacture: numeric only
if "year_of_manufacture" in df.columns:
    df["year_of_manufacture"] = (
        df["year_of_manufacture"]
        .astype(str)
        .str.extract(r"(\d{4})")[0]
        .astype(float)
    )

# 7) Price cleaning:
#   - Examples in your sheet: "₹14.50 Lakh Compare Hyderabad View Seller Details"
#   - Keep only numeric lakh value like "14.50 Lakh", or convert to numeric_lakh
#   - Also normalize variants: "Rs.", "₹", spacing, and remove tails like "Compare ..."

def extract_price_lakh(text):
    """
    Return numeric price in lakh as float if found, else np.nan.
    Accepts patterns like:
      ₹14.50 Lakh, Rs. 7 Lakh, 6.2 Lakh
      46 Lakh, 1.94 Lakh
    Also handles '₹2 Lakh-₹5 Lakh' by taking the first numeric if such rows survived.
    """
    if pd.isna(text):
        return np.nan
    s = str(text)

    # Normalize currency symbols and spaces
    s = s.replace("Rs.", "").replace("Rs", "").replace("₹", "")
    s = re.sub(r"\s+", " ", s).strip()

    # Common case: number followed by 'Lakh'
    m = re.search(r"(\d+(?:\.\d+)?)\s*Lakh", s, flags=re.IGNORECASE)
    if m:
        return float(m.group(1))

    # If only a bare number appears and is large, assume it might be in lakh if explicitly stated elsewhere
    # but to be strict per your instruction, only keep values explicitly in 'Lakh'
    return np.nan

if "price" in df.columns:
    df["price_lakh"] = df["price"].apply(extract_price_lakh)

    # Keep a clean textual form like "14.50 Lakh" only when price_lakh exists
    df["price_clean"] = df["price_lakh"].apply(lambda x: (f"{x:.2f} Lakh") if pd.notna(x) else np.nan)

    # If you want strictly the cleaned text column replacing original 'price'
    df["price"] = df["price_clean"]

    # Optional: also keep a numeric column for analysis
    # df["price_lakh"] already exists

# 8) Remove rows with nulls in critical fields
critical_cols = [c for c in ["car_name", "brand", "model", "kms_driven", "fuel_type", "year_of_manufacture", "price"] if c in df.columns]
if critical_cols:
    df = df.dropna(subset=critical_cols, how="any")

# 9) Strip text fields and standardize casing for categorical columns
text_cols = [c for c in df.columns if df[c].dtype == "object"]
for c in text_cols:
    df[c] = df[c].astype(str).str.strip()

# Optional normalization for fuel_type labels
if "fuel_type" in df.columns:
    df["fuel_type"] = (
        df["fuel_type"]
        .str.title()  # Petrol, Diesel, Cng, Electric, etc.
        .replace({"Cng": "CNG"})  # common fix
    )

# 10) Reorder columns for readability
preferred_order = [col for col in ["car_name", "brand", "model", "year_of_manufacture", "fuel_type", "kms_driven", "price", "price_lakh"] if col in df.columns]
other_cols = [c for c in df.columns if c not in preferred_order]
df = df[preferred_order + other_cols]

# 11) Save outputs
df.to_excel("cardekho_used_cars_nagpur_clean_1.xlsx", index=False)
df.to_csv("cardekho_used_cars_nagpur_clean_1.csv", index=False)
print(f"Rows after cleaning: {len(df)}")


Rows after cleaning: 507


Data Clean Stage 3

In [4]:
import pandas as pd

# Load the cleaned file you created earlier

infile = "cardekho_used_cars_nagpur_clean_1.xlsx"
df = pd.read_excel(infile)

# Drop only the specified columns if they exist
df = df.drop(columns=[c for c in ["price_lakh", "price_clean"] if c in df.columns])

# Save back
df.to_excel("cardekho_used_cars_nagpur_clean_2.xlsx", index=False)
print("Columns now:", list(df.columns))


Columns now: ['car_name', 'brand', 'model', 'year_of_manufacture', 'fuel_type', 'kms_driven', 'price', 'mileage', 'transmission', 'detail_page']


Data Manipulation ; Add Column Location

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

infile = Path("cardekho_used_cars_nagpur_clean_2.xlsx")  # your Excel file
outfile = Path("cardekho_used_cars_nagpur_clean_final.xlsx")

# Read the first sheet (or specify sheet_name="Sheet1" if needed)
# engine openpyxl is used implicitly if installed
df = pd.read_excel(infile, sheet_name=0, engine="openpyxl")

# Ensure we have at least 864 rows
n = min(864, len(df))

# Add/overwrite 'location' only for first n rows
if "location" not in df.columns:
    df["location"] = np.nan
df.loc[:n-1, "location"] = "Nagpur"

# Save back to Excel
with pd.ExcelWriter(outfile, engine="openpyxl") as writer:
    df.to_excel(writer, index=False, sheet_name="Sheet1")

print(f"Loaded {len(df)} rows; set location='Nagpur' for rows 1..{n}; saved to {outfile}")


Loaded 507 rows; set location='Nagpur' for rows 1..507; saved to cardekho_used_cars_nagpur_clean_final.xlsx


C:\Users\shrey\AppData\Local\Temp\ipykernel_30296\726801375.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Nagpur' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:n-1, "location"] = "Nagpur"
